<a href="https://colab.research.google.com/github/siddhapurahet/AI_Concepts_Projects/blob/Interview_questions_answers_project/Interview_Question_Answers_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

==> Installing Dependencies

In [2]:
!pip install langchain langchain-community langchain-core openai pypdf tiktoken aiofiles fastapi uvicorn jinja2 PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


==> Open AI Api Key

In [ ]:
%env OPENAI_API_KEY = "your api key"


==> Taking the raw pdf as input file and extracting the content from the file

In [5]:
from langchain.document_loaders import PyPDFLoader

file_path = "/content/Interview_Question_input_data.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

==> Displaying the content of the input file

In [6]:
data

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.3 (Macintosh)', 'creationdate': '2023-07-19T22:21:03+09:00', 'moddate': '2023-07-19T22:21:07+09:00', 'trapped': '/False', 'source': '/content/Interview_Question_input_data.pdf', 'total_pages': 42, 'page': 0, 'page_label': '1'}, page_content='A Report of the Intergovernmental Panel on Climate Change\nCLIMATE CHANGE 2023\nSynthesis Report\nSummary for Policymakers'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.3 (Macintosh)', 'creationdate': '2023-07-19T22:21:03+09:00', 'moddate': '2023-07-19T22:21:07+09:00', 'trapped': '/False', 'source': '/content/Interview_Question_input_data.pdf', 'total_pages': 42, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.3 (Macintosh)', 'creationdate': '2023-07-19T22:21:03+09:00', 'moddate': '2023-07-19T22:21:07+09:00', 'trapped': '/False', 

==> Extracting the page content from all the content generated. <br>
&nbsp; &nbsp; &nbsp; &nbsp; => The format of the text extracted is : <br> &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;Document(metadata={'metadata': 'producer', 'creator': '', 'creationdate':
'2023-07-19T22:21:03+09:00', 'moddate': '2023-07-19T22:21:07
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;+09:00', 'trapped': '/False', 'source': '/content/
Interview_Question_input_data.pdf', 'total_pages': 42, 'page':
31, 'page_label': '26'}, <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;page_content='26\nSummary for Policymakers\nSummary for Policymakers Delayed mitigation action will further increase global') <br>
      <br>
    But we only want the page content as it has relevant content according to our use case. So we are extracting page_content field from every page throughout the input file.  

In [7]:
question_gen = ""
for page in data:
  question_gen += page.page_content

In [8]:
question_gen
# type(question_gen) --> type = str

'A Report of the Intergovernmental Panel on Climate Change\nCLIMATE CHANGE 2023\nSynthesis Report\nSummary for PolicymakersCLIMATE CHANGE 2023\nSynthesis Report\nSummary for Policymakers\nHoesung Lee (Chair), Katherine Calvin (USA), Dipak Dasgupta (India/USA), Gerhard Krinner (France/Germany), Aditi Mukherji \n(India), Peter Thorne (Ireland/United Kingdom),\xa0Christopher Trisos (South Africa), José Romero (Switzerland), Paulina Aldunce \n(Chile), Ko Barrett (USA), Gabriel Blanco (Argentina), William W. L. Cheung (Canada), Sarah L. Connors (France/United Kingdom), \nFatima Denton (The Gambia), Aïda Diongue-Niang (Senegal), David Dodman (Jamaica/United Kingdom/Netherlands), Matthias \nGarschagen (Germany), Oliver Geden (Germany), Bronwyn Hayward (New Zealand), Christopher Jones (United Kingdom), Frank \nJotzo (Australia), Thelma Krug (Brazil), Rodel Lasco (Philippines), June-Yi Lee (Republic of Korea), Valérie Masson-Delmotte \n(France), Malte Meinshausen (Australia/Germany), Katja Mint

==> Making chunks of the content extracted <br>
&nbsp;&nbsp;&nbsp;&nbsp;=> The need for chunks is because there is specific limit to size of data that can be send to Embedding &model, hence we need to
make smaller chunks of data and then it will be sent to embedding model.

In [9]:
from langchain.text_splitter import TokenTextSplitter

splitter_ques_gen = TokenTextSplitter(
    model_name = "gpt-3.5-turbo",
    chunk_size = 1000,
    chunk_overlap = 100
)

In [10]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)
chunk_ques_gen
# len(chunk_ques_gen)
type(chunk_ques_gen[0])

str

==> Changing the individual chunks from string format to document format <br>
&nbsp;&nbsp;&nbsp;&nbsp;=> It is recommended to pass the document format to the vector embedding model rather than string format. Hence, each chunk_ques_gen is converted to document format.

In [11]:
from langchain.docstore.document import Document

document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

document_ques_gen
type(document_ques_gen[0])

langchain_core.documents.base.Document

==> Passing the document to model to frame it in the Question format <br>
&nbsp;&nbsp;&nbsp;&nbsp;=> Here, we are not using vector database as we only want to frame questions from the document, so a generative AI model would do that, but when generating the answer, we would need to store document into vector database through embedding model. Hence, for answer generation, vector database would play a role as it has to find the most correct answer using searching relevant content from vector database according to the prompt given by user.

==> Using the OpenAI model for generating questions from text

In [15]:
from langchain.chat_models import ChatOpenAI

llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

==> Making a prompt_template which will be given to the model for generating questions

In [17]:
prompt_template = """
You are responsible for creating questions from the text. Make sure that every important question is covered.
You can do this by asking questions about the text below:

-------------
{text}
-------------

Create questions that will make the user more knowledgeble when he completes your Interview.

Questions:
"""

In [18]:
from langchain.prompts import PromptTemplate

Prompt_questions = PromptTemplate(template = prompt_template, input_variables = ['text'])

==> Using a concept of refining template in which the questions generated by the model are again given to the model to verify that the generated questions are proper and not irrelevant. Hence, again it is passed to the model with refine_template that modifies questions if necessary

In [19]:
refine_template = ("""
You are responsible for creating questions from the text. Make sure that every important question is covered.
we have received some practice questions to a certain extent: {existing_answer}.
We have option to refine existing questions or add new ones (only if necessary) with some more context below.

------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, kindly give the original questions.

Questions:
"""
)

In [24]:
refine_prompt_questions = PromptTemplate(
    input_variables = ['existing_answer', 'text'],
    template = refine_template,
)

In [25]:
from langchain.chains.summarize import load_summarize_chain

ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                      chain_type = "refine",
                                      verbose = True,
                                      question_prompt = Prompt_questions,
                                      refine_prompt = refine_prompt_questions

)

In [26]:
questions = ques_gen_chain.run(document_ques_gen)

print(questions)

<ipython-input-26-98adb10d66fd>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  questions = ques_gen_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are responsible for creating questions from the text. Make sure that every important question is covered.
You can do this by asking questions about the text below:

-------------
A Report of the Intergovernmental Panel on Climate Change
CLIMATE CHANGE 2023
Synthesis Report
Summary for PolicymakersCLIMATE CHANGE 2023
Synthesis Report
Summary for Policymakers
Hoesung Lee (Chair), Katherine Calvin (USA), Dipak Dasgupta (India/USA), Gerhard Krinner (France/Germany), Aditi Mukherji 
(India), Peter Thorne (Ireland/United Kingdom), Christopher Trisos (South Africa), José Romero (Switzerland), Paulina Aldunce 
(Chile), Ko Barrett (USA), Gabriel Blanco (Argentina), William W. L. Cheung (Canada), Sarah L. Connors (France/United Kingdom), 
Fatima Denton (The Gambia), Aïda Diongue-Niang (Senegal), David Dodman (Jamaica/United Kingdom/Netherlands), Matthias 
Garschagen (Germany), Oliver G

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}